In [22]:
import pandas as pd
from transformers import BertModel, BertTokenizer

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

In [15]:
df_train = pd.read_json('../../../datasets/中文医疗信息处理评测基准CBLUE/CMeEE/CMeEE_train.json')
df_valid = pd.read_json('../../../datasets/中文医疗信息处理评测基准CBLUE/CMeEE/CMeEE_dev.json')

In [16]:
df_train.head()

,text,entities
0,（5）房室结消融和起搏器植入作为反复发作或难治性心房内折返性心动过速的替代疗法。,"[{'start_idx': 3, 'end_idx': 7, 'type': 'pro',..."
1,（6）发作一次伴血流动力学损害的室性心动过速（ventriculartachycardia）...,"[{'start_idx': 8, 'end_idx': 21, 'type': 'dis'..."
2,4.第三类（1）无症状性WPW综合征患者，年龄小于5岁。,"[{'start_idx': 8, 'end_idx': 17, 'type': 'dis'..."
3,（2）室上性心动过速可用常规抗心律失常药物控制，年龄小于5岁。,"[{'start_idx': 3, 'end_idx': 9, 'type': 'dis',..."
4,（3）非持续性，不考虑为无休止性的阵发性室性心动过速（即一次监视数小时或任何一小时记录的心电...,"[{'start_idx': 3, 'end_idx': 25, 'type': 'dis'..."


In [7]:
pretrained_model = '/home/gechengze/project/pretrained-models/bert-base-chinese/'

model = BertModel.from_pretrained(pretrained_model)
tokenizer = BertTokenizer.from_pretrained(pretrained_model)

Some weights of the model checkpoint at /home/gechengze/project/pretrained-models/bert-base-chinese/ were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
class MyDataset(Dataset):
    def __init__(self, df, tokenizer):
        print('building dataset...')
        
    

In [24]:
entity_types = set()
for enetities in df_train['entities']:
    for entity in enetities:
        entity_types.add(entity['type'])

In [25]:
entity_types

{'bod', 'dep', 'dis', 'dru', 'equ', 'ite', 'mic', 'pro', 'sym'}

In [19]:
tokenizer(df_train['text'][0], add_special_tokens=False)

{'input_ids': [8020, 126, 8021, 2791, 2147, 5310, 3867, 6084, 1469, 6629, 3011, 1690, 3490, 1057, 868, 711, 1353, 1908, 1355, 868, 2772, 7410, 3780, 2595, 2552, 2791, 1079, 2835, 6819, 2595, 2552, 1220, 6814, 6862, 4638, 3296, 807, 4545, 3791, 511], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [20]:
tokenizer(df_train['text'][0])

{'input_ids': [101, 8020, 126, 8021, 2791, 2147, 5310, 3867, 6084, 1469, 6629, 3011, 1690, 3490, 1057, 868, 711, 1353, 1908, 1355, 868, 2772, 7410, 3780, 2595, 2552, 2791, 1079, 2835, 6819, 2595, 2552, 1220, 6814, 6862, 4638, 3296, 807, 4545, 3791, 511, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [5]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(21128, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          